<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.6%20Modelaci%C3%B3n%20supervisada%20y%20no%20supervisada%20de%20Big%20Data/4.6.3%20Regresi%C3%B3n%20Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Regresión Lineal

Se desea examinar un conjunto de datos de clientes de e-commerce para el sitio web y la aplicación móvil de una empresa y se busca construir un modelo de regresión que prediga el gasto anual del cliente en el producto de la empresa.

In [ ]:
generation = "mod4gen<x>"

In [ ]:
data = spark.read.csv(f"/mnt/{generation}/input/ecommerce_customers.csv", inferSchema=True, header=True)

In [ ]:
# Print the Schema of the DataFrame
data.printSchema()

In [ ]:
data.display()

In [ ]:
data.head()

In [ ]:
for item in data.head():
    print(item)

#### VectorAssembler

Se requiere aplicar algunas transformaciones antes de que Spark pueda modelar. Nuestro DataFrame debe contar con dos columnas de la forma `("target", "features")`

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

In [ ]:
assembler = VectorAssembler(inputCols=["Avg Session Length", "Time on App", "Time on Website", "Length of Membership"], outputCol="features")

In [ ]:
type(assembler)

In [ ]:
output = assembler.transform(data)

In [ ]:
output.select("features").show()

In [ ]:
output.display()

In [ ]:
final_data = output.select("features", "Yearly Amount Spent")

In [ ]:
final_data.show()

In [ ]:
train_data, val_data = final_data.randomSplit([0.7, 0.3])

In [ ]:
train_data.describe().show()

In [ ]:
val_data.describe().show()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
# Se instancia el modelo LinearRegression
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [ ]:
type(lr)

In [ ]:
# Ajuste del modelo
lr_model = lr.fit(train_data)

In [ ]:
# Coeficientes y el intercepto de la regresión lineal
print(f"Coefficients: {lr_model.coefficients} Intercept: {lr_model.intercept}")

In [ ]:
# Evaluación del modelo
val_results = lr_model.evaluate(val_data)

In [ ]:
type(val_results)

In [ ]:
val_results.residuals.show()

In [ ]:
unlabeled_data = val_data.select("features")

In [ ]:
predictions = lr_model.transform(unlabeled_data)

In [ ]:
predictions.show()

In [ ]:
val_data.show()

In [ ]:
final_df = val_data.join(predictions, on="features", how="inner")

In [ ]:
final_df.select("Yearly Amount Spent", "prediction").display()

In [ ]:
final_df.select("Yearly Amount Spent", "prediction").display()

In [ ]:
print(f"R2: {val_results.r2}")
print(f"MAE: {val_results.meanAbsoluteError}")
print(f"RMSE: {val_results.rootMeanSquaredError}")